In [ ]:
#ALAN RAMIREZ RIVERA                ALGORITMO GENETICO


import random

def obtener_palabras(n):
    palabras = []
    for i in range(n):
        palabra = input(f"Ingrese la palabra {i+1}: ")
        palabras.append(palabra)
    return palabras

def crear_matriz(palabras):
    longitud_maxima = max(len(palabra) for palabra in palabras)
    matriz = []
    for palabra in palabras:
        fila = list(palabra.ljust(longitud_maxima, '-'))
        matriz.append(fila)
    return matriz

def clonar_matrices(matriz, num_clones):
    matrices_clonadas = [matriz[:] for _ in range(num_clones)]
    return matrices_clonadas

def mutar(matrices):
    for matriz in matrices:
        longitud_maxima = max(len(fila) for fila in matriz)
        posiciones_aleatorias = [random.randint(0, longitud_maxima) for _ in range(len(matriz))]
        matriz_expandida = []
        for i, fila in enumerate(matriz):
            nueva_fila = []
            for j in range(longitud_maxima):
                if j < len(fila):
                    nueva_fila.append(fila[j])
                else:
                    nueva_fila.append(' ')
                if j == posiciones_aleatorias[i]:
                    nueva_fila.insert(j, '-')
            matriz_expandida.append(nueva_fila)
        matriz[:] = matriz_expandida

def imprimir_matriz(matriz):
    for fila in matriz:
        print(fila)

def calcular_puntaje_matriz(matriz, caracteres_a_ignorar=None):
    if caracteres_a_ignorar is None:
        caracteres_a_ignorar = {' ', '-'}
    
    num_columnas_max = max(len(fila) for fila in matriz)
    letras_dic = [dict() for _ in range(num_columnas_max)]  # Crear una lista de diccionarios para cada columna
    contador = [0] * num_columnas_max  # Crear una lista para almacenar el contador de cada columna

    # Iterar sobre cada fila de la matriz
    for fila in matriz:
        # Iterar sobre cada elemento de la fila (columna actual)
        for i, letra in enumerate(fila):
            # Verificar si el carácter debe ser ignorado
            if letra in caracteres_a_ignorar:
                continue  # Saltar al siguiente carácter si es un espacio o un guion
            
            # Verificar si la letra ya está en el diccionario de la columna actual
            if letra in letras_dic[i]:
                if letras_dic[i][letra] == 1:
                    contador[i] += 1  # Incrementar el contador si la letra se repite
                letras_dic[i][letra] += 1  # Continuar contando
            else:
                letras_dic[i][letra] = 1  # Agregar la letra al diccionario si no está presente
    
    # Sumar los contadores de caracteres repetidos en cada columna para obtener el puntaje total
    puntaje_total = sum(contador)
    
    # Devolver el puntaje total
    return puntaje_total


def cruzar_matrices(mejores_matrices):
    hijos = []
    # Tomar parejas de matrices
    for i in range(0, len(mejores_matrices), 2):
        matriz_padre1 = mejores_matrices[i]
        matriz_padre2 = mejores_matrices[i + 1]

        # Dividir las matrices en 3 columnas
        columna1_padre1 = [fila[:len(fila) // 3] for fila in matriz_padre1]
        columna2_padre1 = [fila[len(fila) // 3:2 * len(fila) // 3] for fila in matriz_padre1]
        columna3_padre1 = [fila[2 * len(fila) // 3:] for fila in matriz_padre1]

        columna1_padre2 = [fila[:len(fila) // 3] for fila in matriz_padre2]
        columna2_padre2 = [fila[len(fila) // 3:2 * len(fila) // 3] for fila in matriz_padre2]
        columna3_padre2 = [fila[2 * len(fila) // 3:] for fila in matriz_padre2]

        # Crear hijos combinando los segmentos de los padres
        hijo1 = [columna1_padre1[i] + columna2_padre2[i] + columna3_padre1[i] for i in range(len(columna1_padre1))]
        hijo2 = [columna1_padre2[i] + columna2_padre1[i] + columna3_padre2[i] for i in range(len(columna1_padre2))]

        hijos.extend([hijo1, hijo2])

    return hijos




def eliminar_columnas_de_guiones(matriz):
    columnas_a_eliminar = []

    # Determinar el número de columnas de la matriz
    num_columnas = max(len(fila) for fila in matriz)

    # Iterar sobre cada columna de la matriz
    for j in range(num_columnas):
        columna = [fila[j] for fila in matriz if j < len(fila)]
        # Verificar si la columna está compuesta completamente por guiones o espacios vacíos
        if all(caracter in {'-', ' '} for caracter in columna):
            columnas_a_eliminar.append(j)

    # Eliminar las columnas de guiones de la matriz
    matriz_limpia = []
    for fila in matriz:
        nueva_fila = [fila[j] for j in range(len(fila)) if j not in columnas_a_eliminar]
        matriz_limpia.append(nueva_fila)

    return matriz_limpia



# Obtener palabras
n = int(input("Ingrese el número de palabras: "))
palabras = obtener_palabras(n)

# Crear matriz
matriz_original = crear_matriz(palabras)

# Pedir al usuario el número de clonaciones de la matriz original
num_clones = int(input("Ingrese el número de clonaciones de la matriz original: "))

# Clonar la matriz original
matrices_clonadas = clonar_matrices(matriz_original, num_clones)

# Definir el número de generaciones
num_generaciones = 50 # cambiar esto al número deseado de generaciones



#EL CICLO IMPORTANTE
for generacion in range(num_generaciones):
    print(f"Generación {generacion + 1}:")

    # Mutar la matriz original y las matrices clonadas
    mutar([matriz_original] + matrices_clonadas)

    # Eliminar columnas de guiones de la matriz original y las matrices clonadas
    matriz_original = eliminar_columnas_de_guiones(matriz_original)
    for i, matriz in enumerate(matrices_clonadas):
        matrices_clonadas[i] = eliminar_columnas_de_guiones(matriz)

    # Calcular puntajes y seleccionar las mejores matrices
    puntajes = [calcular_puntaje_matriz(matriz) for matriz in [matriz_original] + matrices_clonadas]
    matrices_ordenadas = sorted(zip([matriz_original] + matrices_clonadas, puntajes), key=lambda x: x[1], reverse=True)
    mejores_matrices = [matriz for matriz, _ in matrices_ordenadas][:4]

    # Cruzar las cuatro mejores matrices
    hijos = cruzar_matrices(mejores_matrices)

    # Imprimir los puntajes de las mejores matrices en esta generación
    print("Puntajes de las mejores matrices:")
    for i, (_, puntaje) in enumerate(matrices_ordenadas[:4]):
        print(f"Matriz {i + 1}: Puntaje = {puntaje}")

    # Imprimir los hijos generados en esta generación
    print("Hijos generados:")
    for i, hijo in enumerate(hijos):
        print(f"Hijo {i + 1}:")
        imprimir_matriz(hijo)
        print()

    # Actualizar la matriz original para la siguiente generación
    matriz_original = mejores_matrices[0]  # Podrías elegir otra estrategia de selección aquí

    print("\n")